# LlamaIndex: Building agent

- Building agent<br>
  https://docs.llamaindex.ai/en/stable/understanding/agent/
  - [Adding other tools](https://docs.llamaindex.ai/en/stable/understanding/agent/tools/)
  - [Maintaining state](https://docs.llamaindex.ai/en/stable/understanding/agent/state/)
  - [Streaming output and events](https://docs.llamaindex.ai/en/stable/understanding/agent/streaming/)
  - [Human in the loop](https://docs.llamaindex.ai/en/stable/understanding/agent/human_in_the_loop/)
  - [Multi-agent systems with AgentWorkflow](https://docs.llamaindex.ai/en/stable/understanding/agent/multi_agent/)<br>
- Tutorial repo
  - https://github.com/run-llama/python-agents-tutorial

## SETUP

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables (for API key)
load_dotenv()

# Set up OpenAI API key
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("Please set the OPENAI_API_KEY environment variable or add it to a .env file")

# Define the model to use
MODEL_GPT = "gpt-4o-mini"

## SETUP (LlamaIndex)

In [2]:
# !pip install llama-index-core llama-index-llms-openai python-dotenv

In [3]:
from dotenv import load_dotenv

load_dotenv()

from llama_index.llms.openai import OpenAI
from llama_index.core.agent.workflow import AgentWorkflow
from llama_index.core.agent.workflow import FunctionAgent

## Create basic tools

In [4]:
def multiply(a: float, b: float) -> float:
    """Multiply two numbers and returns the product"""
    return a * b

def add(a: float, b: float) -> float:
    """Add two numbers and returns the sum"""
    return a + b

## Initialize the LLM

In [5]:
llm = OpenAI(model="gpt-4o-mini")

## Initialize the agent

In [6]:
workflow = FunctionAgent(
    name="Agent",
    description="Useful for basic mathematical operations",
    tools=[multiply, add],
    llm=llm,
    system_prompt="You are an agent that can perform basic mathematical operations using tools.",
)

## Ask a question

In [7]:
# async def main():
#     response = await workflow.run(user_msg="What is 20+(2*4)?")
#     print(response)

# if __name__ == "__main__":
#     import asyncio

#     asyncio.run(main())

In [8]:
response = await workflow.run(user_msg="What is 20+(2*4)?")
print(response)

The result of \( 20 + (2 \times 4) \) is 28.


## Adding other tools

### Using an existing tool from LlamaHub

**Yahoo Finance tool from LlamaHub**
- https://llamahub.ai/l/tools/llama-index-tools-yahoo-finance?from=tools

In [9]:
# !pip install llama-index-tools-yahoo-finance

In [10]:
from llama_index.tools.yahoo_finance import YahooFinanceToolSpec

In [11]:
finance_tools = YahooFinanceToolSpec().to_tool_list()

In [12]:
finance_tools.extend([multiply, add])

In [13]:
workflow = FunctionAgent(
    name="Agent",
    description="Useful for performing financial operations.",
    llm=OpenAI(model="gpt-4o-mini"),
    tools=finance_tools,
    system_prompt="You are a helpful assistant.",
)

# async def main():
#     response = await workflow.run(
#         user_msg="What's the current stock price of NVIDIA?"
#     )
#     print(response)

In [14]:
response = await workflow.run(user_msg="What's the current stock price of NVIDIA?")
print(response)

The current stock price of NVIDIA (NVDA) is $111.43.


In [15]:
response = await workflow.run(user_msg="What's the current stock price of GOLD?")
print(response)

The current stock price of Barrick Gold Corporation (ticker: GOLD) is **$19.56**.


This is cheating a little bit, because our model already knew the ticker symbol for NVIDIA.<br>
If it were a less well-known corporation you would need to add a search tool like **Tavily** to find the ticker symbol.
- https://llamahub.ai/l/tools/llama-index-tools-tavily-research

### Building and contributing your own tools

Example of what the code of the Yahoo finance tool looks like:<br>
[GitHub - LlamaIndex Integrations](https://github.com/run-llama/llama_index/blob/main/llama-index-integrations/tools/llama-index-tools-yahoo-finance/llama_index/tools/yahoo_finance/base.py)
- A class that extends BaseToolSpec
- A set of arbitrary Python functions
- A spec_functions list that maps the functions to the tool's API<br>

## Maintaining state

In [18]:
# from llama_index.core.workflow import Context

# ctx = Context(workflow)

# ERROR: AttributeError: 'FunctionAgent' object has no attribute '_get_steps'

In [19]:
from llama_index.core.agent.workflow import AgentWorkflow

workflow = AgentWorkflow.from_tools_or_functions(
    [multiply, add],
    llm=llm,
    system_prompt="You are an agent that can perform basic mathematical operations using tools.",
)

In [20]:
from llama_index.core.workflow import Context

In [21]:
ctx = Context(workflow)

In [22]:
response = await workflow.run(user_msg="Hi, my name is Laurie!", ctx=ctx)
print(response)

Hello Laurie! How can I assist you today?


In [23]:
response2 = await workflow.run(user_msg="What's my name?", ctx=ctx)
print(response2)

Your name is Laurie!


### Maintaining state over longer periods

In [24]:
from llama_index.core.workflow import JsonPickleSerializer, JsonSerializer

In [25]:
ctx_dict = ctx.to_dict(serializer=JsonSerializer())

In [26]:
restored_ctx = Context.from_dict(
    workflow, ctx_dict, serializer=JsonSerializer()
)

In [27]:
response3 = await workflow.run(user_msg="What's my name?", ctx=restored_ctx)

In [28]:
print(response3)

Your name is Laurie.


### Tools and state

In [29]:
async def set_name(ctx: Context, name: str) -> str:
    state = await ctx.get("state")
    state["name"] = name
    await ctx.set("state", state)
    return f"Name set to {name}"

In [30]:
workflow = AgentWorkflow.from_tools_or_functions(
    [set_name],
    llm=llm,
    system_prompt="You are a helpful assistant that can set a name.",
    initial_state={"name": "unset"},
)

In [31]:
ctx = Context(workflow)

# check if it knows a name before setting it
response4 = await workflow.run(user_msg="What's my name?", ctx=ctx)
print(str(response4))

Your name has been set to "unset."


In [32]:
response5 = await workflow.run(user_msg="My name is Laurie", ctx=ctx)
print(str(response5))

Your name has been updated to "Laurie."


In [33]:
state = await ctx.get("state")
print("Name as stored in state: ", state["name"])

Name as stored in state:  Laurie


## Streaming output and events

Create a new tool that takes some time to execute.<br>
In this case we'll use a web search tool called **Tavily**, which is available in LlamaHub.
- https://llamahub.ai/l/tools/llama-index-tools-tavily-research

In [40]:
# !pip install llama-index-tools-tavily-research

In [41]:
from llama_index.tools.tavily_research import TavilyToolSpec
import os

In [42]:
tavily_tool = TavilyToolSpec(api_key=os.getenv("TAVILY_API_KEY"))

In [43]:
workflow = FunctionAgent(
    name="Agent",
    description="Useful for search the web for information",
    tools=tavily_tool.to_tool_list(),
    llm=llm,
    system_prompt="You're a helpful assistant that can search the web for information.",
)

In [44]:
from llama_index.core.agent.workflow import AgentStream

In [45]:
handler = workflow.run(user_msg="What's the weather like in San Francisco?")

async for event in handler.stream_events():
    if isinstance(event, AgentStream):
        print(event.delta, end="", flush=True)

The current weather in San Francisco is as follows:

- **Temperature**: 12.8°C (55°F)
- **Condition**: Partly cloudy
- **Wind**: 9.4 mph (15.1 kph) from the southwest
- **Humidity**: 83%
- **Cloud Cover**: 75%
- **Visibility**: 16 km (9 miles)

It feels like 11.4°C (52.4°F) due to wind chill. There is no precipitation reported at this time.

For more details, you can check the full weather report [here](https://www.weatherapi.com/).

In [46]:
handler = workflow.run(user_msg="What's the weather like in Prague?")

async for event in handler.stream_events():
    if isinstance(event, AgentStream):
        print(event.delta, end="", flush=True)

The current weather in Prague is as follows:

- **Temperature**: 6.1°C (43°F)
- **Condition**: Mist
- **Feels Like**: 5.5°C (41.8°F)
- **Wind**: 2.9 mph (4.7 kph) from the East-Northeast
- **Humidity**: 81%
- **Visibility**: 5 km (3 miles)
- **Pressure**: 1015 mb (29.97 inHg)

The forecast for today indicates a high of 12°C (54°F) and a low of 2°C (36°F), with a chance of patchy rain.

For more details, you can check the full weather report [here](https://www.timeanddate.com/weather/czech-republic/prague).

AgentStream is just one of many events that AgentWorkflow emits as it runs.<br>
The others are:
- **AgentInput**: the full message object that begins the agent's execution
- **AgentOutput**: the response from the agent
- **ToolCall**: which tools were called and with what arguments
- **ToolCallResult**: the result of a tool call